In [1]:
!pip install datasets>=1.18.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 30.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 31.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 13.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tokenizers: filename=tokenizers-0.10.3-cp310-cp310-linux_x86_64.whl size=2898404 sha256=a37153a34b2ef54a7ca3d5c951daa40c3069053aa0c1a96f2ee01cd971b4fa4a
  Stored in directory: /home/users1/tranti/.cache/pip/wheels/aa/86/dd/ce32c5dc1d4f430e3b2728de777f50c8e48c16577606d4f337
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=26e6c3f687c613d23c9a58b5a97d26c498c1c5cfcbed59ca4292b2b6c593bb69
  Stored in directory: /home/users1/tranti/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42f

In [17]:
from datasets import load_dataset, Dataset, Audio, load_from_disk
import numpy as np
import json
import os
import re
test_dataset = load_from_disk("../timit_test_dataset")


In [10]:
type(test_dataset[1]["input_values"])
input_values = np.array(test_dataset[10]["input_values"])

signal_power = np.mean(input_values**2)
print(signal_power)
noise_power_db = 10
noise_amp_linear = 10**(-noise_power_db/20)
awgn_noise = np.random.normal(size=input_values.shape, scale=noise_amp_linear)
input_values = input_values + awgn_noise
signal_power_noise = np.mean(awgn_noise**2)
print(f"signal power = {signal_power}, noise power: {signal_power_noise}")
SNR = 10*np.log10(signal_power/signal_power_noise)
print(SNR)



0.9976127457147286
signal power = 0.9976127457147286, noise power: 0.10003171169122875
9.988242888527875


In [11]:
import IPython.display as ipd
import numpy as np

rand_int =1

print(test_dataset[rand_int]["text"])
ipd.Audio(data=input_values, autoplay=True, rate=16000)

don't ask me to carry an oily rag like that


In [12]:
def add_awgn_noise_to_signal(input_signal : np.ndarray, noise_power_dB : float):
    noise_amp_linear = 10**(-noise_power_dB/20)
    awgn_noise = np.random.normal(size=input_signal.shape, scale=noise_amp_linear)

    return input_signal + awgn_noise

noise_power_list = [-10,0,10,20]

def add_noise(batch):
    # batched output is "un-batched" to ensure mapping is correct
    input_values = np.array(batch["input_values"])

    for noise_power in noise_power_list:
        noisy_signal = add_awgn_noise_to_signal(input_values, noise_power)
        batch[f"input_values_noisy_{int(noise_power)}"] = noisy_signal

    return batch

noise_test_dataset = test_dataset.map(add_noise)

In [13]:
print(noise_test_dataset[1]["text"])
ipd.Audio(data=noise_test_dataset[1]["input_values_noisy_0"], autoplay=True, rate=16000)

don't ask me to carry an oily rag like that


In [15]:
noise_test_dataset.save_to_disk("/mount/arbeitsdaten/studenten1/team-lab-phonetics/2023/student_directories/tran/asr_timit/speech_datasets/timit_test_dataset_noisy")